In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd
import warnings
from sklearn.neighbors import BallTree
import lightgbm as lgb
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)
input_path = '../input/'
gdf_land = gpd.read_file(os.path.join(input_path, 'L02-25.geojson'))
df_land = gdf_land.copy()
df_land['price'] = df_land['L02_008']
df = pd.read_csv(os.path.join(input_path, 'train.csv'))
test = pd.read_csv(os.path.join(input_path, 'test.csv'))
gdf_land.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21431 entries, 0 to 21430
Columns: 140 entries, L02_001 to geometry
dtypes: float64(4), geometry(1), int32(69), object(66)
memory usage: 17.2+ MB


In [18]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

def preprocess_data(input_df, df_land, is_train=True):
    """
    データフレームを受け取り、特徴量エンジニアリングを行って返す関数
    
    Args:
        input_df (pd.DataFrame): 処理したいデータ (train or test)
        df_land (pd.DataFrame): 国土数値情報の地価データ (lat, lon, price列必須)
        is_train (bool): 学習用データかどうか (Trueなら目的変数を作る)
    
    Returns:
        df (pd.DataFrame): 処理後のデータ
        features (list): 学習に使う特徴量のリスト
        target (pd.Series or None): 学習用の場合、変換済みの目的変数 (対数単価)
    """
    # 元のデータを壊さないようにコピー
    df = input_df.copy()
    if is_train:
        # 例えば「200㎡以上」は外れ値として捨てる
        # (分布を見て閾値は調整してください。一旦200にしておきます)
        df = df[df['unit_area'] < 2000]
    # -----------------------------------------------------
    # 1. 築年数の計算 (yyyymm -> 月数)
    # -----------------------------------------------------
    # 文字列にして日付型に変換
    df['temp_time'] = pd.to_datetime(df['year_built'].astype(str), format='%Y%m', errors='coerce')
    
    # 基準日（2025年12月など）からの月数を計算
    base_date = pd.to_datetime('2025-12-01')
    # 日数 ÷ 30.44 で「月数」にする (float)
    df['building_month'] = (base_date - df['temp_time']).dt.days / 30.44
    df['building_month'] = df['building_month'].astype(float) # 念のため型変換

    # -----------------------------------------------------
    # 2. 国土数値情報（地価）の結合 (BallTree)
    # -----------------------------------------------------
    # 地価データの準備 (NaN削除 & ラジアン変換)
    df_land['lat'] = df_land.geometry.y
    df_land['lon'] = df_land.geometry.x
    land_clean = df_land[['lat', 'lon', 'price']].dropna()
    land_rad = np.deg2rad(land_clean[['lat', 'lon']])
    input_rad = np.deg2rad(df[['lat', 'lon']])
    
    # BallTree構築 (metric='haversine' で地球の丸みを考慮)
    # ※毎回作ると少し重いので、本来は関数の外で作って渡すのがベストですが、今回はここでやります
    tree = BallTree(land_rad, metric='haversine')
    
    # 最寄り検索 (k=1)
    dists, indices = tree.query(input_rad, k=1)
    
    # 結合
    df['land_price'] = land_clean['price'].values[indices.flatten()]
    df['dist_to_land_price'] = dists.flatten() * 6371 * 1000 # メートル変換
    
    # 地価の対数変換 (特徴量として使いやすくする)
    df['log_land_price'] = np.log1p(df['land_price'])

    # -----------------------------------------------------
    # 3. カテゴリ変数の処理
    # -----------------------------------------------------
    cat_cols = ['addr1_2', 'layout', 'direction', 'structure'] # 必要に応じて追加
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).fillna('unknown').astype('category')
    
    # -----------------------------------------------------
    # 4. 特徴量リストの定義
    # -----------------------------------------------------
    # 数値特徴量 + カテゴリ特徴量
    num_features = ['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month']
    features = num_features + [c for c in cat_cols if c in df.columns]

    # -----------------------------------------------------
    # 5. 目的変数の作成 (Trainのみ)
    # -----------------------------------------------------
    target = None
    if is_train:
        # 単価を計算して対数変換 (MAPE対策)
        # room_price_per_area がなければ money_room / unit_area で作る
        if 'room_price_per_area' not in df.columns:
             # money_roomがある前提
             current_price = df['money_room']
        else:
             current_price = df['room_price_per_area'] * df['unit_area'] # 一旦総額に戻すなど調整必要かも

        # 今回のコンペに合わせて調整：
        # 「単価」を予測ターゲットにする場合
        unit_price = df['money_room'] / df['unit_area']
        target = np.log1p(unit_price)

    return df, features, target

In [31]:
# ---------------------------------------------------------
# 1. 前処理 (Train & Test)
# ---------------------------------------------------------
print("前処理を実行中...")

# Trainデータの処理 (学習用なので is_train=True)
# ※ preprocess_data関数はさっき定義した最新版を使ってください
train_processed, features, target_log = preprocess_data(df, df_land, is_train=True)
# 【Train独自の処理】
# 面積がないデータは「教師として質が悪い」ので、行ごと消す
train_processed = train_processed.dropna(subset=['unit_area'])

# 削除した分、target（目的変数）も合わせる必要があるので、再計算するか
# あるいは preprocess_data の中で dropna してから target を作るのが安全ですが、
# 今の段階なら「埋めるための値（中央値）」をここで計算して保存しておくのが大事です。

# ★Testを埋めるための「正解の値」をキープ！
fill_value = train_processed['unit_area'].median() 


# ---------------------------------------------------------
# 2. Testデータの処理
# ---------------------------------------------------------
# 先に関数を通す
test_processed, _, _ = preprocess_data(test, df_land, is_train=False)

# 【Test独自の処理】
# 提出用なので行は消せない。「Trainの中央値」を使って埋める
test_processed['unit_area'] = test_processed['unit_area'].fillna(fill_value)
print(f"使用する特徴量: {len(features)}個")
print(features)

# ---------------------------------------------------------
# 2. モデル学習
# ---------------------------------------------------------
print("\nモデル学習中...")

# 検証用に分割 (提出前に手元のスコアを知るため)
X = train_processed[features]
y = target_log
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 学習実行
model = lgb.LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

# ---------------------------------------------------------
# 3. 予測 & 結果の復元
# ---------------------------------------------------------
print("予測を実行中...")

# テストデータに対して予測 (結果は対数単価)
pred_log = model.predict(test_processed[features])

# 【重要】対数を戻す (log -> exp)
pred_price_per_m2 = np.expm1(pred_log)

# 【重要】単価を総額に戻す (単価 × 面積)
# ※ 提出は「総額」のはずなので、必ず面積を掛けます
pred_total_price = pred_price_per_m2 * test_processed['unit_area']

# ---------------------------------------------------------
# 4. 提出用CSVの作成
# ---------------------------------------------------------
print("CSV作成中...")

# id列と予測値をくっつける
submission = pd.DataFrame({
    'id': test['id'],         # テストデータのID
    'prediction': pred_total_price # 予測した総額
})

# CSVに書き出し (header=Falseが必要な場合が多いので注意)
# 今回は「id,値段」の形式ということなので、ヘッダーありかなしか確認してください。
# 多くの場合、SIGNATEなどはヘッダー不要(header=False)か、指定のヘッダー名が必要です。
# ここでは一旦、一般的な「ヘッダーなし」で保存します。
submission.to_csv('submission.csv', index=False, header=False)

print("\n完了！ 'submission.csv' が保存されました。")
print("先頭5行はこんな感じです↓")
print(submission.head())

前処理を実行中...


使用する特徴量: 5個
['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'addr1_2']

モデル学習中...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 641
[LightGBM] [Info] Number of data points in the train set: 253483, number of used features: 4
[LightGBM] [Info] Start training from score 12.477169
予測を実行中...
CSV作成中...

完了！ 'submission.csv' が保存されました。
先頭5行はこんな感じです↓
   id    prediction
0   0  2.011500e+07
1   1  3.089160e+07
2   2  1.904898e+07
3   3  2.070388e+07
4   4  1.465635e+07


In [28]:
# テストデータの欠損状況を確認
cols_to_check = ['unit_area', 'lat', 'lon', 'year_built']
print(test[cols_to_check].isnull().sum())

# 欠損している行だけ見てみる
print(test[test['unit_area'].isnull()])

unit_area     11390
lat               0
lon               0
year_built     4819
dtype: int64
            id  target_ym  building_id  building_status building_create_date  \
7            7     202301       179871                1  2014-06-27 21:09:54   
8            8     202301       148834                1  2014-06-27 19:24:14   
13          13     202301       158994                1  2014-06-27 21:11:23   
16          16     202301        81076                1  2014-06-27 21:11:25   
31          31     202301       160922                1  2014-06-27 21:11:29   
...        ...        ...          ...              ...                  ...   
112397  112397     202307        82207                1  2014-06-27 21:09:11   
112398  112398     202307        54983                1  2014-06-27 21:09:11   
112401  112401     202307         1505                1  2014-06-27 21:09:11   
112430  112430     202307       125919                1  2014-06-27 21:09:18   
112433  112433     202307  